# Installation and Repository cloning

In [1]:
#!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
#!pip install adversarial-robustness-toolbox -U
#!pip install git+https://github.com/RobustBench/robustbench.git
#!pip install multiprocess
#!pip install matplotlib
#!pip install pillow
#!pip install foolbox
#!pip install numba

## Import Libraries

In [2]:
import torch
import utils
import adversarial_attack.attack_utils as attack_utils

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/home/shao/anaconda3/envs/art/lib/python3.12/site-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
/home/shao/anaconda3/envs/art/lib/python3.12/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/home/shao/anaconda3/envs/art/lib/python3.12/site-packages/timm/models/helpers.py:7: FutureWarning: Importing from timm.models.helpers is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)


## Load the dataset

In [3]:
dataset = 'cifar10' #'cifar10' #'imagenet'
splitsize =1000 # full, int: splitsize
xtest, ytest = utils.load_dataset(dataset= dataset, dataset_split=splitsize)

Files already downloaded and verified


## Load and Prepare Adversarial Model

In [4]:
model = 'CroceL1' # MainiAVG, MainiMSD, CroceL1, Wang2023Better_WRN-28-10, Wang2023Better_WRN-28-10, Salman2020Do_R18(Imagenet)
net, art_net, fb_net, alias = utils.get_model(dataset= dataset, modelname=model,norm='L2') # specify the norm in case of robustbench models

no input normalization


/home/shao/elastic_net_adversarial_attack/utils.py:58: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load('./models/pretrained_models/CroceL1.pth', map_location

## Test Accuracy

In [5]:
utils.test_accuracy(net, xtest, ytest)
print(ytest)


Accuracy of the test set is: 84.000%

tensor([8, 1, 8, 2, 4, 4, 9, 8, 4, 5, 8, 3, 6, 1, 3, 8, 5, 7, 4, 2, 5, 4, 4, 2,
        1, 2, 9, 9, 1, 4, 9, 0, 3, 1, 7, 3, 8, 4, 0, 3, 4, 2, 8, 8, 5, 3, 7, 7,
        1, 9, 9, 2, 5, 6, 0, 7, 4, 1, 0, 8, 2, 9, 3, 6, 6, 3, 4, 8, 6, 7, 3, 3,
        4, 1, 1, 0, 1, 9, 5, 1, 2, 9, 0, 2, 8, 2, 3, 6, 2, 7, 6, 8, 5, 7, 1, 7,
        3, 7, 5, 9])


## Parameters 

In [6]:
Experiment = attack_utils.Experiment_class(art_net, fb_net, net, xtest, ytest, alias,
                        epsilon_l1 = 12, 
                        epsilon_l2= 0.5, 
                        eps_iter = 0.1,  
                        norm = 1,  # 1, 2, np.inf
                        max_iterations =300, 
                        verbose=False)

# Hyperparameter Sweep Experiment

In [7]:
#hyperparameter sweep. Pick only one model.
attack_type = 'exp_attack_l1'
hyperparameter_range=[2.0]
hyperparameter = 'learning_rate' #'beta'

results_dict_hyperparameter_sweep = Experiment.hyperparameter_sweep(hyperparameter=hyperparameter, range=hyperparameter_range, attack_type=attack_type)

		-------------- Hyperparameter Sweep for Attack: exp_attack_l1: learning_rate = 2.0 ----------------

20 images done. Current Adversarial Accuracy (L1 / L2/ EN): 40.000000% / 65.000000%/40.000000%
60 images done. Current Adversarial Accuracy (L1 / L2/ EN): 51.666667% / 68.333333%/51.666667%
80 images done. Current Adversarial Accuracy (L1 / L2/ EN): 55.000000% / 70.000000%/55.000000%
100 images done. Current Adversarial Accuracy (L1 / L2/ EN): 60.000000% / 73.000000%/60.000000%

Adversarial accuracy (L1 / L2/ EN): 60.00 / 73.00/ 60.00%


average sparsity: 54.86%

learning_rate2.0 attack success rate in epsilon (L1 / L2):  28.5714  /  13.0952
mean adv. distance (L1 / L2):  11.8917  /  0.8373
Evaluation results are saved under "./data/hyperparameter_sweep_exp_attack_l1_CroceL1.json".


In [ ]:
#Attack comparison
attack_types = [#'fast_gradient_method',
                #'projected_gradient_descent',
                #'pgd_early_stopping',
                #'deep_fool',
                #'brendel_bethge',
                #'carlini_wagner_l2',
                #'elastic_net',
                #'exp_attack',
                #'exp_attack_smooth',
                #'exp_attack_l1_l2',
                #'auto_projected_gradient_descent',
                #'elastic_net_L1_rule',
                #'elastic_net_L1_rule_higher_beta',
                #'ART_AutoAttack',
                #'original_AutoAttack',
                'exp_attack_l1',
                'custom_apgd'
                ]

results_dict_attack_comparison = Experiment.attack_comparison(attack_types)